In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn torch joblib

# Data analysis and Prep.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import joblib

In [ ]:
dataset = pd.read_csv('dataset.csv')

In [ ]:
import random
random.seed(48)

In [ ]:
dataset.head()

,segment,anomaly,train,channel,sampling,duration,len,mean,var,std,...,smooth10_n_peaks,smooth20_n_peaks,diff_peaks,diff2_peaks,diff_var,diff2_var,gaps_squared,len_weighted,var_div_duration,var_div_len
0,1,1,1,CADC0872,1,279,280,8.533143e-07,3.494283e-10,0.000019,...,3,2,4,6,1.271176e-10,2.960666e-10,309,280,1.252431e-12,1.247958e-12
1,2,1,1,CADC0872,1,476,477,-3.639396e-06,6.476485e-10,0.000025,...,1,1,5,8,1.489383e-12,3.004752e-12,644,477,1.360606e-12,1.357754e-12
2,3,1,1,CADC0872,1,594,595,1.170788e-05,5.592877e-10,0.000024,...,2,2,2,3,4.112280e-12,1.029918e-11,772,595,9.415618e-13,9.399794e-13
3,4,1,1,CADC0872,1,271,272,8.486808e-07,5.466024e-10,0.000023,...,2,2,3,6,2.475760e-11,6.240985e-11,339,272,2.016983e-12,2.009568e-12
4,5,0,0,CADC0872,1,255,257,1.058485e-05,5.279023e-10,0.000023,...,1,1,78,87,5.547101e-13,7.035422e-13,357,257,2.070205e-12,2.054094e-12


In [ ]:
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2123 entries, 0 to 2122
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   segment           2123 non-null   int64  
 1   anomaly           2123 non-null   int64  
 2   train             2123 non-null   int64  
 3   channel           2123 non-null   object 
 4   sampling          2123 non-null   int64  
 5   duration          2123 non-null   int64  
 6   len               2123 non-null   int64  
 7   mean              2123 non-null   float64
 8   var               2123 non-null   float64
 9   std               2123 non-null   float64
 10  kurtosis          2123 non-null   float64
 11  skew              2123 non-null   float64
 12  n_peaks           2123 non-null   int64  
 13  smooth10_n_peaks  2123 non-null   int64  
 14  smooth20_n_peaks  2123 non-null   int64  
 15  diff_peaks        2123 non-null   int64  
 16  diff2_peaks       2123 non-null   int64  


In [ ]:
dataset_corr = dataset.drop(['channel'], axis = 1)
dataset_corr.corr()

,segment,anomaly,train,sampling,duration,len,mean,var,std,kurtosis,...,smooth10_n_peaks,smooth20_n_peaks,diff_peaks,diff2_peaks,diff_var,diff2_var,gaps_squared,len_weighted,var_div_duration,var_div_len
segment,1.000000,-0.133887,-0.009139,0.594096,-0.178043,-0.441953,-0.095082,-0.072263,-0.036008,0.048778,...,-0.175961,0.166357,-0.397870,-0.389892,0.063709,0.058242,0.127228,-0.184845,-0.035786,-0.012237
anomaly,-0.133887,1.000000,-0.013118,-0.267742,0.169642,0.293134,-0.161937,-0.185838,-0.173805,0.233905,...,0.527219,0.132104,-0.155948,-0.162844,0.108702,0.153694,0.118749,0.156882,-0.162620,-0.127807
train,-0.009139,-0.013118,1.000000,0.005411,-0.006108,-0.019249,0.005138,-0.008735,-0.002043,-0.008455,...,-0.040088,-0.061003,0.012081,0.015388,-0.002131,0.000042,0.015970,-0.005731,-0.009633,-0.007363
sampling,0.594096,-0.267742,0.005411,1.000000,-0.166177,-0.791557,0.174819,0.080157,0.093727,-0.095661,...,-0.250290,0.057668,-0.559012,-0.566322,0.161295,0.068812,0.324512,-0.144708,0.095240,0.298552
duration,-0.178043,0.169642,-0.006108,-0.166177,1.000000,0.595424,0.031889,0.173474,0.158733,0.102165,...,0.269853,-0.153259,0.342381,0.406555,-0.098506,-0.060116,0.427955,0.996448,-0.011609,-0.014797
len,-0.441953,0.293134,-0.019249,-0.791557,0.595424,1.000000,-0.127975,-0.027096,-0.034696,0.126388,...,0.380322,0.029282,0.595271,0.640647,-0.144525,-0.065919,-0.126699,0.577777,-0.104478,-0.235295
mean,-0.095082,-0.161937,0.005138,0.174819,0.031889,-0.127975,1.000000,0.660945,0.737551,0.001821,...,-0.137341,-0.233126,-0.175325,-0.145729,0.319710,0.131407,0.044103,0.032451,0.702944,0.661473
var,-0.072263,-0.185838,-0.008735,0.080157,0.173474,-0.027096,0.660945,1.000000,0.966811,0.052461,...,-0.139595,-0.233402,-0.150496,-0.103837,0.253804,0.087045,0.083367,0.171687,0.903313,0.788910
std,-0.036008,-0.173805,-0.002043,0.093727,0.158733,-0.034696,0.737551,0.966811,1.000000,0.070435,...,-0.152962,-0.269941,-0.173050,-0.124226,0.283795,0.111032,0.076783,0.153972,0.894202,0.782000
kurtosis,0.048778,0.233905,-0.008455,-0.095661,0.102165,0.126388,0.001821,0.052461,0.070435,1.000000,...,0.197549,-0.023397,-0.054897,-0.054359,0.115051,0.099874,0.076696,0.088471,0.057473,0.051430


In [ ]:
print(dataset.duplicated()[dataset.duplicated() == True].count())

0


In [ ]:
print(dataset.isnull().sum())

segment             0
anomaly             0
train               0
channel             0
sampling            0
duration            0
len                 0
mean                0
var                 0
std                 0
kurtosis            0
skew                0
n_peaks             0
smooth10_n_peaks    0
smooth20_n_peaks    0
diff_peaks          0
diff2_peaks         0
diff_var            0
diff2_var           0
gaps_squared        0
len_weighted        0
var_div_duration    0
var_div_len         0
dtype: int64


In [ ]:
print(dataset.value_counts("anomaly"))

anomaly
0    1689
1     434
Name: count, dtype: int64


In [ ]:
features = [
    "mean", "var", "std", "len", "duration", "len_weighted", "gaps_squared", "n_peaks",
    "smooth10_n_peaks", "smooth20_n_peaks", "var_div_duration", "var_div_len",
    "diff_peaks", "diff2_peaks", "diff_var", "diff2_var", "kurtosis", "skew",
]

In [ ]:
X_train, y_train = dataset.loc[dataset.train==1, features], dataset.loc[dataset.train==1, "anomaly"]
X_test, y_test = dataset.loc[dataset.train==0, features], dataset.loc[dataset.train==0, "anomaly"]
X_train_nominal = dataset.loc[(dataset.anomaly==0)&(dataset.train==1), features]

prep = StandardScaler()
X_train_nominal2 = prep.fit_transform(X_train_nominal)
X_train2 = prep.transform(X_train)
X_test2 = prep.transform(X_test)

# FCNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train = dataset.loc[dataset.train==1, features]
y_train = dataset.loc[dataset.train==1, "anomaly"]

X_test = dataset.loc[dataset.train==0, features]
y_test = dataset.loc[dataset.train==0, "anomaly"]

X_train_nominal = dataset.loc[(dataset.anomaly==0)&(dataset.train==1), features]

prep = StandardScaler()
X_train_nominal2 = prep.fit_transform(X_train_nominal)
X_train2 = prep.transform(X_train)
X_test2 = prep.transform(X_test)

X_train_tensor = torch.tensor(X_train2, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test2, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class FCNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout):
        super(FCNN, self).__init__()
        layers = []
        in_dim = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(in_dim, h))
            layers.append(nn.BatchNorm1d(h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            in_dim = h
        layers.append(nn.Linear(in_dim, 1))
        layers.append(nn.Sigmoid())  # binary classification
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [ ]:
# Hyperparameters
input_dim = X_train_tensor.shape[1]
hidden_dims = [128, 64, 32]
dropout = 0.2
lr = 0.001
epochs = 75

In [ ]:
model = FCNN(input_dim=input_dim, hidden_dims=hidden_dims, dropout=dropout)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    all_preds = []
    all_labels = []

    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        preds_class = (preds.detach().cpu().numpy() > 0.5).astype(int)
        all_preds.extend(preds_class)
        all_labels.extend(yb.detach().cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, zero_division=0)
    rec = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}, "
          f"Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for xb, yb in test_loader:
        preds = model(xb)
        preds_class = (preds.cpu().numpy() > 0.5).astype(int)
        all_preds.extend(preds_class)
        all_labels.extend(yb.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, zero_division=0)
rec = recall_score(all_labels, all_preds, zero_division=0)
f1 = f1_score(all_labels, all_preds, zero_division=0)

print(f"\nTest Metrics -> Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")


Epoch 1/75, Loss: 0.5582, Acc: 0.8124, Prec: 0.5333, Rec: 0.5483, F1: 0.5407
Epoch 2/75, Loss: 0.4455, Acc: 0.9040, Prec: 0.9286, Rec: 0.5670, F1: 0.7041
Epoch 3/75, Loss: 0.3754, Acc: 0.9040, Prec: 0.9421, Rec: 0.5576, F1: 0.7006
Epoch 4/75, Loss: 0.3209, Acc: 0.9122, Prec: 0.9289, Rec: 0.6106, F1: 0.7368
Epoch 5/75, Loss: 0.2836, Acc: 0.9141, Prec: 0.9220, Rec: 0.6262, F1: 0.7458
Epoch 6/75, Loss: 0.2457, Acc: 0.9266, Prec: 0.9016, Rec: 0.7134, F1: 0.7965
Epoch 7/75, Loss: 0.2218, Acc: 0.9360, Prec: 0.9041, Rec: 0.7632, F1: 0.8277
Epoch 8/75, Loss: 0.2012, Acc: 0.9366, Prec: 0.9135, Rec: 0.7570, F1: 0.8279
Epoch 9/75, Loss: 0.1987, Acc: 0.9329, Prec: 0.8639, Rec: 0.7913, F1: 0.8260
Epoch 10/75, Loss: 0.1897, Acc: 0.9391, Prec: 0.8916, Rec: 0.7944, F1: 0.8402
Epoch 11/75, Loss: 0.1827, Acc: 0.9379, Prec: 0.8936, Rec: 0.7850, F1: 0.8358
Epoch 12/75, Loss: 0.1605, Acc: 0.9442, Prec: 0.9028, Rec: 0.8100, F1: 0.8539
Epoch 13/75, Loss: 0.1450, Acc: 0.9523, Prec: 0.9298, Rec: 0.8255, F1: 0.

In [ ]:
model = torch.jit.load("fcnn_model.pt", map_location="cpu")
scripted_model = torch.jit.script(model)
scripted_model.save("fcnn_model.pt")
joblib.dump(prep, "scaler.pkl")

['scaler.pkl']

In [ ]:
"""
Epoch 1/75, Loss: 0.6888, Acc: 0.5326, Prec: 0.2740, Rec: 0.8006, F1: 0.4083
Epoch 2/75, Loss: 0.5124, Acc: 0.8582, Prec: 0.6518, Rec: 0.6355, F1: 0.6435
Epoch 3/75, Loss: 0.4383, Acc: 0.8946, Prec: 0.8283, Rec: 0.6012, F1: 0.6968
Epoch 4/75, Loss: 0.3774, Acc: 0.9059, Prec: 0.8904, Rec: 0.6075, F1: 0.7222
Epoch 5/75, Loss: 0.3235, Acc: 0.9109, Prec: 0.8714, Rec: 0.6542, F1: 0.7473
Epoch 6/75, Loss: 0.2946, Acc: 0.9172, Prec: 0.8795, Rec: 0.6822, F1: 0.7684
Epoch 7/75, Loss: 0.2632, Acc: 0.9210, Prec: 0.8679, Rec: 0.7165, F1: 0.7850
Epoch 8/75, Loss: 0.2450, Acc: 0.9253, Prec: 0.8976, Rec: 0.7103, F1: 0.7930
Epoch 9/75, Loss: 0.2201, Acc: 0.9304, Prec: 0.8646, Rec: 0.7757, F1: 0.8177
Epoch 10/75, Loss: 0.2242, Acc: 0.9266, Prec: 0.8723, Rec: 0.7445, F1: 0.8034
Epoch 11/75, Loss: 0.2092, Acc: 0.9316, Prec: 0.8759, Rec: 0.7695, F1: 0.8192
Epoch 12/75, Loss: 0.1899, Acc: 0.9373, Prec: 0.9018, Rec: 0.7726, F1: 0.8322
Epoch 13/75, Loss: 0.1852, Acc: 0.9454, Prec: 0.9091, Rec: 0.8100, F1: 0.8567
Epoch 14/75, Loss: 0.1759, Acc: 0.9442, Prec: 0.9000, Rec: 0.8131, F1: 0.8543
Epoch 15/75, Loss: 0.1743, Acc: 0.9366, Prec: 0.8819, Rec: 0.7913, F1: 0.8342
Epoch 16/75, Loss: 0.1732, Acc: 0.9460, Prec: 0.9304, Rec: 0.7913, F1: 0.8552
Epoch 17/75, Loss: 0.1526, Acc: 0.9435, Prec: 0.8997, Rec: 0.8100, F1: 0.8525
Epoch 18/75, Loss: 0.1765, Acc: 0.9366, Prec: 0.8793, Rec: 0.7944, F1: 0.8347
Epoch 19/75, Loss: 0.1603, Acc: 0.9442, Prec: 0.8893, Rec: 0.8255, F1: 0.8562
Epoch 20/75, Loss: 0.1463, Acc: 0.9517, Prec: 0.9150, Rec: 0.8380, F1: 0.8748
Epoch 21/75, Loss: 0.1613, Acc: 0.9486, Prec: 0.9283, Rec: 0.8069, F1: 0.8633
Epoch 22/75, Loss: 0.1517, Acc: 0.9460, Prec: 0.8930, Rec: 0.8318, F1: 0.8613
Epoch 23/75, Loss: 0.1470, Acc: 0.9517, Prec: 0.9266, Rec: 0.8255, F1: 0.8731
Epoch 24/75, Loss: 0.1493, Acc: 0.9467, Prec: 0.9041, Rec: 0.8224, F1: 0.8613
Epoch 25/75, Loss: 0.1498, Acc: 0.9467, Prec: 0.8986, Rec: 0.8287, F1: 0.8622
Epoch 26/75, Loss: 0.1368, Acc: 0.9523, Prec: 0.9268, Rec: 0.8287, F1: 0.8750
Epoch 27/75, Loss: 0.1387, Acc: 0.9555, Prec: 0.9139, Rec: 0.8598, F1: 0.8860
Epoch 28/75, Loss: 0.1408, Acc: 0.9492, Prec: 0.9027, Rec: 0.8380, F1: 0.8691
Epoch 29/75, Loss: 0.1250, Acc: 0.9605, Prec: 0.9358, Rec: 0.8629, F1: 0.8979
Epoch 30/75, Loss: 0.1337, Acc: 0.9561, Prec: 0.9373, Rec: 0.8380, F1: 0.8849
Epoch 31/75, Loss: 0.1451, Acc: 0.9492, Prec: 0.9027, Rec: 0.8380, F1: 0.8691
Epoch 32/75, Loss: 0.1385, Acc: 0.9529, Prec: 0.9301, Rec: 0.8287, F1: 0.8764
Epoch 33/75, Loss: 0.1398, Acc: 0.9504, Prec: 0.9231, Rec: 0.8224, F1: 0.8699
Epoch 34/75, Loss: 0.1220, Acc: 0.9567, Prec: 0.9118, Rec: 0.8692, F1: 0.8900
Epoch 35/75, Loss: 0.1317, Acc: 0.9511, Prec: 0.9204, Rec: 0.8287, F1: 0.8721
Epoch 36/75, Loss: 0.1369, Acc: 0.9523, Prec: 0.8990, Rec: 0.8598, F1: 0.8790
Epoch 37/75, Loss: 0.1399, Acc: 0.9536, Prec: 0.9273, Rec: 0.8349, F1: 0.8787
Epoch 38/75, Loss: 0.1292, Acc: 0.9536, Prec: 0.9186, Rec: 0.8442, F1: 0.8799
Epoch 39/75, Loss: 0.1295, Acc: 0.9536, Prec: 0.9186, Rec: 0.8442, F1: 0.8799
Epoch 40/75, Loss: 0.1258, Acc: 0.9555, Prec: 0.9167, Rec: 0.8567, F1: 0.8857
Epoch 41/75, Loss: 0.1308, Acc: 0.9567, Prec: 0.9228, Rec: 0.8567, F1: 0.8885
Epoch 42/75, Loss: 0.1368, Acc: 0.9523, Prec: 0.9268, Rec: 0.8287, F1: 0.8750
Epoch 43/75, Loss: 0.1361, Acc: 0.9561, Prec: 0.9343, Rec: 0.8411, F1: 0.8852
Epoch 44/75, Loss: 0.1284, Acc: 0.9523, Prec: 0.9097, Rec: 0.8474, F1: 0.8774
Epoch 45/75, Loss: 0.1243, Acc: 0.9605, Prec: 0.9216, Rec: 0.8785, F1: 0.8995
Epoch 46/75, Loss: 0.1246, Acc: 0.9517, Prec: 0.9150, Rec: 0.8380, F1: 0.8748
Epoch 47/75, Loss: 0.1257, Acc: 0.9542, Prec: 0.9218, Rec: 0.8442, F1: 0.8813
Epoch 48/75, Loss: 0.1203, Acc: 0.9580, Prec: 0.9262, Rec: 0.8598, F1: 0.8918
Epoch 49/75, Loss: 0.1302, Acc: 0.9567, Prec: 0.9228, Rec: 0.8567, F1: 0.8885
Epoch 50/75, Loss: 0.1321, Acc: 0.9536, Prec: 0.9076, Rec: 0.8567, F1: 0.8814
Epoch 51/75, Loss: 0.1244, Acc: 0.9542, Prec: 0.9079, Rec: 0.8598, F1: 0.8832
Epoch 52/75, Loss: 0.1145, Acc: 0.9580, Prec: 0.9379, Rec: 0.8474, F1: 0.8903
Epoch 53/75, Loss: 0.1119, Acc: 0.9598, Prec: 0.9356, Rec: 0.8598, F1: 0.8961
Epoch 54/75, Loss: 0.1279, Acc: 0.9529, Prec: 0.9271, Rec: 0.8318, F1: 0.8768
Epoch 55/75, Loss: 0.1181, Acc: 0.9573, Prec: 0.9259, Rec: 0.8567, F1: 0.8900
Epoch 56/75, Loss: 0.1181, Acc: 0.9548, Prec: 0.9192, Rec: 0.8505, F1: 0.8835
Epoch 57/75, Loss: 0.1174, Acc: 0.9573, Prec: 0.9439, Rec: 0.8380, F1: 0.8878
Epoch 58/75, Loss: 0.1149, Acc: 0.9580, Prec: 0.9320, Rec: 0.8536, F1: 0.8911
Epoch 59/75, Loss: 0.1252, Acc: 0.9548, Prec: 0.9082, Rec: 0.8629, F1: 0.8850
Epoch 60/75, Loss: 0.1264, Acc: 0.9536, Prec: 0.9215, Rec: 0.8411, F1: 0.8795
Epoch 61/75, Loss: 0.1219, Acc: 0.9580, Prec: 0.9410, Rec: 0.8442, F1: 0.8900
Epoch 62/75, Loss: 0.1119, Acc: 0.9636, Prec: 0.9398, Rec: 0.8754, F1: 0.9065
Epoch 63/75, Loss: 0.1261, Acc: 0.9529, Prec: 0.9100, Rec: 0.8505, F1: 0.8792
Epoch 64/75, Loss: 0.1042, Acc: 0.9624, Prec: 0.9454, Rec: 0.8629, F1: 0.9023
Epoch 65/75, Loss: 0.1087, Acc: 0.9624, Prec: 0.9454, Rec: 0.8629, F1: 0.9023
Epoch 66/75, Loss: 0.1056, Acc: 0.9624, Prec: 0.9485, Rec: 0.8598, F1: 0.9020
Epoch 67/75, Loss: 0.1184, Acc: 0.9555, Prec: 0.9281, Rec: 0.8442, F1: 0.8842
Epoch 68/75, Loss: 0.0949, Acc: 0.9611, Prec: 0.9331, Rec: 0.8692, F1: 0.9000
Epoch 69/75, Loss: 0.1135, Acc: 0.9617, Prec: 0.9362, Rec: 0.8692, F1: 0.9015
Epoch 70/75, Loss: 0.1004, Acc: 0.9624, Prec: 0.9365, Rec: 0.8723, F1: 0.9032
Epoch 71/75, Loss: 0.1072, Acc: 0.9668, Prec: 0.9408, Rec: 0.8910, F1: 0.9152
Epoch 72/75, Loss: 0.1146, Acc: 0.9617, Prec: 0.9483, Rec: 0.8567, F1: 0.9002
Epoch 73/75, Loss: 0.1126, Acc: 0.9592, Prec: 0.9354, Rec: 0.8567, F1: 0.8943
Epoch 74/75, Loss: 0.0966, Acc: 0.9630, Prec: 0.9456, Rec: 0.8660, F1: 0.9041
Epoch 75/75, Loss: 0.1035, Acc: 0.9617, Prec: 0.9452, Rec: 0.8598, F1: 0.9005

Test Metrics -> Acc: 0.9754, Prec: 0.9630, Rec: 0.9204, F1: 0.9412


batch_size = 64
input_dim = X_train_tensor.shape[1]
hidden_dims = [128, 64, 32]
dropout = 0.3
lr = 0.001
epochs = 75

8 saniye
"""

In [ ]:
"""
Epoch 1/75, Loss: 0.5449, Acc: 0.8394, Prec: 0.6484, Rec: 0.4424, F1: 0.5259
Epoch 2/75, Loss: 0.4098, Acc: 0.8990, Prec: 0.9444, Rec: 0.5296, F1: 0.6786
Epoch 3/75, Loss: 0.3364, Acc: 0.9009, Prec: 0.9358, Rec: 0.5452, F1: 0.6890
Epoch 4/75, Loss: 0.2953, Acc: 0.9109, Prec: 0.9497, Rec: 0.5888, F1: 0.7269
Epoch 5/75, Loss: 0.2561, Acc: 0.9159, Prec: 0.9390, Rec: 0.6231, F1: 0.7491
Epoch 6/75, Loss: 0.2339, Acc: 0.9228, Prec: 0.9231, Rec: 0.6729, F1: 0.7784
Epoch 7/75, Loss: 0.2154, Acc: 0.9285, Prec: 0.8996, Rec: 0.7259, F1: 0.8034
Epoch 8/75, Loss: 0.1950, Acc: 0.9348, Prec: 0.8974, Rec: 0.7632, F1: 0.8249
Epoch 9/75, Loss: 0.1976, Acc: 0.9366, Prec: 0.8873, Rec: 0.7850, F1: 0.8331
Epoch 10/75, Loss: 0.1787, Acc: 0.9354, Prec: 0.9098, Rec: 0.7539, F1: 0.8245
Epoch 11/75, Loss: 0.1627, Acc: 0.9498, Prec: 0.9141, Rec: 0.8287, F1: 0.8693
Epoch 12/75, Loss: 0.1647, Acc: 0.9473, Prec: 0.9129, Rec: 0.8162, F1: 0.8618
Epoch 13/75, Loss: 0.1560, Acc: 0.9454, Prec: 0.9062, Rec: 0.8131, F1: 0.8571
Epoch 14/75, Loss: 0.1522, Acc: 0.9473, Prec: 0.9158, Rec: 0.8131, F1: 0.8614
Epoch 15/75, Loss: 0.1484, Acc: 0.9473, Prec: 0.9044, Rec: 0.8255, F1: 0.8632
Epoch 16/75, Loss: 0.1501, Acc: 0.9454, Prec: 0.9179, Rec: 0.8006, F1: 0.8552
Epoch 17/75, Loss: 0.1499, Acc: 0.9548, Prec: 0.9338, Rec: 0.8349, F1: 0.8816
Epoch 18/75, Loss: 0.1418, Acc: 0.9498, Prec: 0.9141, Rec: 0.8287, F1: 0.8693
Epoch 19/75, Loss: 0.1332, Acc: 0.9561, Prec: 0.9373, Rec: 0.8380, F1: 0.8849
Epoch 20/75, Loss: 0.1358, Acc: 0.9523, Prec: 0.9181, Rec: 0.8380, F1: 0.8762
Epoch 21/75, Loss: 0.1229, Acc: 0.9592, Prec: 0.9414, Rec: 0.8505, F1: 0.8936
Epoch 22/75, Loss: 0.1289, Acc: 0.9555, Prec: 0.9371, Rec: 0.8349, F1: 0.8830
Epoch 23/75, Loss: 0.1233, Acc: 0.9548, Prec: 0.9109, Rec: 0.8598, F1: 0.8846
Epoch 24/75, Loss: 0.1198, Acc: 0.9580, Prec: 0.9536, Rec: 0.8318, F1: 0.8885
Epoch 25/75, Loss: 0.1324, Acc: 0.9536, Prec: 0.8997, Rec: 0.8660, F1: 0.8825
Epoch 26/75, Loss: 0.1206, Acc: 0.9598, Prec: 0.9573, Rec: 0.8380, F1: 0.8937
Epoch 27/75, Loss: 0.1222, Acc: 0.9567, Prec: 0.9118, Rec: 0.8692, F1: 0.8900
Epoch 28/75, Loss: 0.1144, Acc: 0.9617, Prec: 0.9643, Rec: 0.8411, F1: 0.8985
Epoch 29/75, Loss: 0.1111, Acc: 0.9592, Prec: 0.9295, Rec: 0.8629, F1: 0.8950
Epoch 30/75, Loss: 0.1176, Acc: 0.9630, Prec: 0.9517, Rec: 0.8598, F1: 0.9034
Epoch 31/75, Loss: 0.1100, Acc: 0.9636, Prec: 0.9369, Rec: 0.8785, F1: 0.9068
Epoch 32/75, Loss: 0.1099, Acc: 0.9617, Prec: 0.9514, Rec: 0.8536, F1: 0.8998
Epoch 33/75, Loss: 0.1228, Acc: 0.9586, Prec: 0.9264, Rec: 0.8629, F1: 0.8935
Epoch 34/75, Loss: 0.1125, Acc: 0.9611, Prec: 0.9450, Rec: 0.8567, F1: 0.8987
Epoch 35/75, Loss: 0.1053, Acc: 0.9624, Prec: 0.9454, Rec: 0.8629, F1: 0.9023
Epoch 36/75, Loss: 0.1112, Acc: 0.9592, Prec: 0.9324, Rec: 0.8598, F1: 0.8947
Epoch 37/75, Loss: 0.1018, Acc: 0.9624, Prec: 0.9547, Rec: 0.8536, F1: 0.9013
Epoch 38/75, Loss: 0.1130, Acc: 0.9586, Prec: 0.9474, Rec: 0.8411, F1: 0.8911
Epoch 39/75, Loss: 0.1067, Acc: 0.9611, Prec: 0.9191, Rec: 0.8847, F1: 0.9016
Epoch 40/75, Loss: 0.1020, Acc: 0.9617, Prec: 0.9248, Rec: 0.8816, F1: 0.9027
Epoch 41/75, Loss: 0.1085, Acc: 0.9649, Prec: 0.9649, Rec: 0.8567, F1: 0.9076
Epoch 42/75, Loss: 0.1155, Acc: 0.9580, Prec: 0.9349, Rec: 0.8505, F1: 0.8907
Epoch 43/75, Loss: 0.1033, Acc: 0.9592, Prec: 0.9414, Rec: 0.8505, F1: 0.8936
Epoch 44/75, Loss: 0.1055, Acc: 0.9567, Prec: 0.9257, Rec: 0.8536, F1: 0.8882
Epoch 45/75, Loss: 0.1014, Acc: 0.9611, Prec: 0.9576, Rec: 0.8442, F1: 0.8974
Epoch 46/75, Loss: 0.1026, Acc: 0.9580, Prec: 0.9320, Rec: 0.8536, F1: 0.8911
Epoch 47/75, Loss: 0.1038, Acc: 0.9642, Prec: 0.9459, Rec: 0.8723, F1: 0.9076
Epoch 48/75, Loss: 0.1027, Acc: 0.9611, Prec: 0.9390, Rec: 0.8629, F1: 0.8994
Epoch 49/75, Loss: 0.0993, Acc: 0.9636, Prec: 0.9488, Rec: 0.8660, F1: 0.9055
Epoch 50/75, Loss: 0.1148, Acc: 0.9630, Prec: 0.9367, Rec: 0.8754, F1: 0.9050
Epoch 51/75, Loss: 0.1085, Acc: 0.9592, Prec: 0.9324, Rec: 0.8598, F1: 0.8947
Epoch 52/75, Loss: 0.0955, Acc: 0.9661, Prec: 0.9556, Rec: 0.8723, F1: 0.9121
Epoch 53/75, Loss: 0.1027, Acc: 0.9611, Prec: 0.9390, Rec: 0.8629, F1: 0.8994
Epoch 54/75, Loss: 0.0936, Acc: 0.9668, Prec: 0.9408, Rec: 0.8910, F1: 0.9152
Epoch 55/75, Loss: 0.0996, Acc: 0.9592, Prec: 0.9384, Rec: 0.8536, F1: 0.8940
Epoch 56/75, Loss: 0.0971, Acc: 0.9655, Prec: 0.9586, Rec: 0.8660, F1: 0.9100
Epoch 57/75, Loss: 0.0981, Acc: 0.9592, Prec: 0.9414, Rec: 0.8505, F1: 0.8936
Epoch 58/75, Loss: 0.0999, Acc: 0.9636, Prec: 0.9256, Rec: 0.8910, F1: 0.9079
Epoch 59/75, Loss: 0.1002, Acc: 0.9624, Prec: 0.9365, Rec: 0.8723, F1: 0.9032
Epoch 60/75, Loss: 0.0871, Acc: 0.9668, Prec: 0.9467, Rec: 0.8847, F1: 0.9147
Epoch 61/75, Loss: 0.1016, Acc: 0.9592, Prec: 0.9507, Rec: 0.8411, F1: 0.8926
Epoch 62/75, Loss: 0.0960, Acc: 0.9655, Prec: 0.9404, Rec: 0.8847, F1: 0.9117
Epoch 63/75, Loss: 0.0921, Acc: 0.9674, Prec: 0.9498, Rec: 0.8847, F1: 0.9161
Epoch 64/75, Loss: 0.0964, Acc: 0.9649, Prec: 0.9461, Rec: 0.8754, F1: 0.9094
Epoch 65/75, Loss: 0.0922, Acc: 0.9680, Prec: 0.9500, Rec: 0.8879, F1: 0.9179
Epoch 66/75, Loss: 0.0890, Acc: 0.9674, Prec: 0.9439, Rec: 0.8910, F1: 0.9167
Epoch 67/75, Loss: 0.0847, Acc: 0.9661, Prec: 0.9525, Rec: 0.8754, F1: 0.9123
Epoch 68/75, Loss: 0.0967, Acc: 0.9605, Prec: 0.9272, Rec: 0.8723, F1: 0.8989
Epoch 69/75, Loss: 0.1017, Acc: 0.9617, Prec: 0.9422, Rec: 0.8629, F1: 0.9008
Epoch 70/75, Loss: 0.0942, Acc: 0.9668, Prec: 0.9437, Rec: 0.8879, F1: 0.9149
Epoch 71/75, Loss: 0.0855, Acc: 0.9699, Prec: 0.9596, Rec: 0.8879, F1: 0.9223
Epoch 72/75, Loss: 0.0835, Acc: 0.9711, Prec: 0.9568, Rec: 0.8972, F1: 0.9260
Epoch 73/75, Loss: 0.0921, Acc: 0.9649, Prec: 0.9316, Rec: 0.8910, F1: 0.9108
Epoch 74/75, Loss: 0.0981, Acc: 0.9686, Prec: 0.9593, Rec: 0.8816, F1: 0.9188
Epoch 75/75, Loss: 0.0986, Acc: 0.9617, Prec: 0.9392, Rec: 0.8660, F1: 0.9011

Test Metrics -> Acc: 0.9811, Prec: 0.9640, Rec: 0.9469, F1: 0.9554

batch_size = 64
input_dim = X_train_tensor.shape[1]
hidden_dims = [128, 64, 32]
dropout = 0.2
lr = 0.001
epochs = 75

8 saniye

"""

In [ ]:
"""
Epoch 1/75, Loss: 0.4933, Acc: 0.8689, Prec: 0.8684, Rec: 0.4112, F1: 0.5581
Epoch 2/75, Loss: 0.3614, Acc: 0.8965, Prec: 0.8861, Rec: 0.5576, F1: 0.6845
Epoch 3/75, Loss: 0.2887, Acc: 0.9216, Prec: 0.9336, Rec: 0.6573, F1: 0.7715
Epoch 4/75, Loss: 0.2466, Acc: 0.9335, Prec: 0.8826, Rec: 0.7726, F1: 0.8239
Epoch 5/75, Loss: 0.2225, Acc: 0.9272, Prec: 0.8622, Rec: 0.7601, F1: 0.8079
Epoch 6/75, Loss: 0.1944, Acc: 0.9442, Prec: 0.9056, Rec: 0.8069, F1: 0.8534
Epoch 7/75, Loss: 0.1828, Acc: 0.9398, Prec: 0.9032, Rec: 0.7850, F1: 0.8400
Epoch 8/75, Loss: 0.1700, Acc: 0.9460, Prec: 0.9038, Rec: 0.8193, F1: 0.8595
Epoch 9/75, Loss: 0.1664, Acc: 0.9442, Prec: 0.9113, Rec: 0.8006, F1: 0.8524
Epoch 10/75, Loss: 0.1553, Acc: 0.9467, Prec: 0.8960, Rec: 0.8318, F1: 0.8627
Epoch 11/75, Loss: 0.1559, Acc: 0.9460, Prec: 0.9181, Rec: 0.8037, F1: 0.8571
Epoch 12/75, Loss: 0.1411, Acc: 0.9523, Prec: 0.9125, Rec: 0.8442, F1: 0.8770
Epoch 13/75, Loss: 0.1289, Acc: 0.9548, Prec: 0.9136, Rec: 0.8567, F1: 0.8842
Epoch 14/75, Loss: 0.1356, Acc: 0.9536, Prec: 0.9103, Rec: 0.8536, F1: 0.8810
Epoch 15/75, Loss: 0.1327, Acc: 0.9542, Prec: 0.9247, Rec: 0.8411, F1: 0.8809
Epoch 16/75, Loss: 0.1346, Acc: 0.9511, Prec: 0.9324, Rec: 0.8162, F1: 0.8704
Epoch 17/75, Loss: 0.1387, Acc: 0.9511, Prec: 0.9037, Rec: 0.8474, F1: 0.8746
Epoch 18/75, Loss: 0.1368, Acc: 0.9542, Prec: 0.9276, Rec: 0.8380, F1: 0.8805
Epoch 19/75, Loss: 0.1333, Acc: 0.9555, Prec: 0.9340, Rec: 0.8380, F1: 0.8834
Epoch 20/75, Loss: 0.1240, Acc: 0.9548, Prec: 0.9308, Rec: 0.8380, F1: 0.8820
Epoch 21/75, Loss: 0.1270, Acc: 0.9523, Prec: 0.8964, Rec: 0.8629, F1: 0.8794
Epoch 22/75, Loss: 0.1153, Acc: 0.9605, Prec: 0.9448, Rec: 0.8536, F1: 0.8969
Epoch 23/75, Loss: 0.1177, Acc: 0.9586, Prec: 0.9264, Rec: 0.8629, F1: 0.8935
Epoch 24/75, Loss: 0.1209, Acc: 0.9555, Prec: 0.9252, Rec: 0.8474, F1: 0.8846
Epoch 25/75, Loss: 0.1157, Acc: 0.9624, Prec: 0.9547, Rec: 0.8536, F1: 0.9013
Epoch 26/75, Loss: 0.1169, Acc: 0.9555, Prec: 0.9371, Rec: 0.8349, F1: 0.8830
Epoch 27/75, Loss: 0.1029, Acc: 0.9617, Prec: 0.9362, Rec: 0.8692, F1: 0.9015
Epoch 28/75, Loss: 0.1066, Acc: 0.9573, Prec: 0.9288, Rec: 0.8536, F1: 0.8896
Epoch 29/75, Loss: 0.1126, Acc: 0.9611, Prec: 0.9609, Rec: 0.8411, F1: 0.8970
Epoch 30/75, Loss: 0.1129, Acc: 0.9548, Prec: 0.9082, Rec: 0.8629, F1: 0.8850
Epoch 31/75, Loss: 0.1078, Acc: 0.9630, Prec: 0.9517, Rec: 0.8598, F1: 0.9034
Epoch 32/75, Loss: 0.1033, Acc: 0.9586, Prec: 0.9412, Rec: 0.8474, F1: 0.8918
Epoch 33/75, Loss: 0.1040, Acc: 0.9655, Prec: 0.9493, Rec: 0.8754, F1: 0.9109
Epoch 34/75, Loss: 0.1101, Acc: 0.9674, Prec: 0.9590, Rec: 0.8754, F1: 0.9153
Epoch 35/75, Loss: 0.1012, Acc: 0.9592, Prec: 0.9354, Rec: 0.8567, F1: 0.8943
Epoch 36/75, Loss: 0.1047, Acc: 0.9624, Prec: 0.9336, Rec: 0.8754, F1: 0.9035
Epoch 37/75, Loss: 0.1042, Acc: 0.9611, Prec: 0.9481, Rec: 0.8536, F1: 0.8984
Epoch 38/75, Loss: 0.0946, Acc: 0.9636, Prec: 0.9398, Rec: 0.8754, F1: 0.9065
Epoch 39/75, Loss: 0.0999, Acc: 0.9617, Prec: 0.9577, Rec: 0.8474, F1: 0.8992
Epoch 40/75, Loss: 0.0995, Acc: 0.9598, Prec: 0.9241, Rec: 0.8723, F1: 0.8974
Epoch 41/75, Loss: 0.1038, Acc: 0.9592, Prec: 0.9507, Rec: 0.8411, F1: 0.8926
Epoch 42/75, Loss: 0.1072, Acc: 0.9642, Prec: 0.9430, Rec: 0.8754, F1: 0.9079
Epoch 43/75, Loss: 0.0910, Acc: 0.9668, Prec: 0.9527, Rec: 0.8785, F1: 0.9141
Epoch 44/75, Loss: 0.0969, Acc: 0.9636, Prec: 0.9340, Rec: 0.8816, F1: 0.9071
Epoch 45/75, Loss: 0.0951, Acc: 0.9636, Prec: 0.9488, Rec: 0.8660, F1: 0.9055
Epoch 46/75, Loss: 0.0973, Acc: 0.9655, Prec: 0.9404, Rec: 0.8847, F1: 0.9117
Epoch 47/75, Loss: 0.1033, Acc: 0.9611, Prec: 0.9360, Rec: 0.8660, F1: 0.8997
Epoch 48/75, Loss: 0.0989, Acc: 0.9642, Prec: 0.9583, Rec: 0.8598, F1: 0.9064
Epoch 49/75, Loss: 0.0903, Acc: 0.9649, Prec: 0.9373, Rec: 0.8847, F1: 0.9103
Epoch 50/75, Loss: 0.0836, Acc: 0.9705, Prec: 0.9757, Rec: 0.8754, F1: 0.9228
Epoch 51/75, Loss: 0.0805, Acc: 0.9755, Prec: 0.9519, Rec: 0.9252, F1: 0.9384
Epoch 52/75, Loss: 0.0978, Acc: 0.9661, Prec: 0.9652, Rec: 0.8629, F1: 0.9112
Epoch 53/75, Loss: 0.0912, Acc: 0.9693, Prec: 0.9387, Rec: 0.9065, F1: 0.9223
Epoch 54/75, Loss: 0.0929, Acc: 0.9636, Prec: 0.9488, Rec: 0.8660, F1: 0.9055
Epoch 55/75, Loss: 0.0998, Acc: 0.9624, Prec: 0.9307, Rec: 0.8785, F1: 0.9038
Epoch 56/75, Loss: 0.0868, Acc: 0.9655, Prec: 0.9375, Rec: 0.8879, F1: 0.9120
Epoch 57/75, Loss: 0.0952, Acc: 0.9630, Prec: 0.9396, Rec: 0.8723, F1: 0.9047
Epoch 58/75, Loss: 0.0843, Acc: 0.9711, Prec: 0.9568, Rec: 0.8972, F1: 0.9260
Epoch 59/75, Loss: 0.0993, Acc: 0.9611, Prec: 0.9302, Rec: 0.8723, F1: 0.9003
Epoch 60/75, Loss: 0.0900, Acc: 0.9711, Prec: 0.9568, Rec: 0.8972, F1: 0.9260
Epoch 61/75, Loss: 0.0834, Acc: 0.9636, Prec: 0.9228, Rec: 0.8941, F1: 0.9082
Epoch 62/75, Loss: 0.0747, Acc: 0.9724, Prec: 0.9601, Rec: 0.9003, F1: 0.9293
Epoch 63/75, Loss: 0.0875, Acc: 0.9649, Prec: 0.9585, Rec: 0.8629, F1: 0.9082
Epoch 64/75, Loss: 0.0921, Acc: 0.9655, Prec: 0.9236, Rec: 0.9034, F1: 0.9134
Epoch 65/75, Loss: 0.0863, Acc: 0.9686, Prec: 0.9656, Rec: 0.8754, F1: 0.9183
Epoch 66/75, Loss: 0.0908, Acc: 0.9699, Prec: 0.9505, Rec: 0.8972, F1: 0.9231
Epoch 67/75, Loss: 0.0830, Acc: 0.9674, Prec: 0.9381, Rec: 0.8972, F1: 0.9172
Epoch 68/75, Loss: 0.0739, Acc: 0.9730, Prec: 0.9455, Rec: 0.9190, F1: 0.9321
Epoch 69/75, Loss: 0.0899, Acc: 0.9642, Prec: 0.9286, Rec: 0.8910, F1: 0.9094
Epoch 70/75, Loss: 0.0866, Acc: 0.9686, Prec: 0.9625, Rec: 0.8785, F1: 0.9186
Epoch 71/75, Loss: 0.0995, Acc: 0.9642, Prec: 0.9430, Rec: 0.8754, F1: 0.9079
Epoch 72/75, Loss: 0.0804, Acc: 0.9705, Prec: 0.9477, Rec: 0.9034, F1: 0.9250
Epoch 73/75, Loss: 0.0954, Acc: 0.9636, Prec: 0.9458, Rec: 0.8692, F1: 0.9058
Epoch 74/75, Loss: 0.0888, Acc: 0.9668, Prec: 0.9351, Rec: 0.8972, F1: 0.9157
Epoch 75/75, Loss: 0.0778, Acc: 0.9711, Prec: 0.9450, Rec: 0.9097, F1: 0.9270

Test Metrics -> Acc: 0.9792, Prec: 0.9554, Rec: 0.9469, F1: 0.9511


batch_size=64
input_dim = X_train_tensor.shape[1]
hidden_dims = [256,128, 64, 32]
dropout = 0.2
lr = 0.001
epochs = 75

10 saniye
"""

In [ ]:
"""
Epoch 1/75, Loss: 0.5538, Acc: 0.8350, Prec: 0.5954, Rec: 0.5639, F1: 0.5792
Epoch 2/75, Loss: 0.4377, Acc: 0.9009, Prec: 0.9137, Rec: 0.5607, F1: 0.6950
Epoch 3/75, Loss: 0.3529, Acc: 0.9260, Prec: 0.8889, Rec: 0.7227, F1: 0.7973
Epoch 4/75, Loss: 0.3016, Acc: 0.9279, Prec: 0.8732, Rec: 0.7508, F1: 0.8074
Epoch 5/75, Loss: 0.2627, Acc: 0.9360, Prec: 0.8982, Rec: 0.7695, F1: 0.8289
Epoch 6/75, Loss: 0.2209, Acc: 0.9479, Prec: 0.9020, Rec: 0.8318, F1: 0.8655
Epoch 7/75, Loss: 0.2072, Acc: 0.9404, Prec: 0.9036, Rec: 0.7882, F1: 0.8419
Epoch 8/75, Loss: 0.1933, Acc: 0.9442, Prec: 0.9000, Rec: 0.8131, F1: 0.8543
Epoch 9/75, Loss: 0.1711, Acc: 0.9498, Prec: 0.9085, Rec: 0.8349, F1: 0.8701
Epoch 10/75, Loss: 0.1726, Acc: 0.9467, Prec: 0.9214, Rec: 0.8037, F1: 0.8586
Epoch 11/75, Loss: 0.1651, Acc: 0.9504, Prec: 0.9116, Rec: 0.8349, F1: 0.8715
Epoch 12/75, Loss: 0.1657, Acc: 0.9454, Prec: 0.9091, Rec: 0.8100, F1: 0.8567
Epoch 13/75, Loss: 0.1432, Acc: 0.9536, Prec: 0.9303, Rec: 0.8318, F1: 0.8783
Epoch 14/75, Loss: 0.1443, Acc: 0.9504, Prec: 0.9116, Rec: 0.8349, F1: 0.8715
Epoch 15/75, Loss: 0.1358, Acc: 0.9548, Prec: 0.9278, Rec: 0.8411, F1: 0.8824
Epoch 16/75, Loss: 0.1358, Acc: 0.9573, Prec: 0.9439, Rec: 0.8380, F1: 0.8878
Epoch 17/75, Loss: 0.1310, Acc: 0.9580, Prec: 0.9410, Rec: 0.8442, F1: 0.8900
Epoch 18/75, Loss: 0.1451, Acc: 0.9542, Prec: 0.9161, Rec: 0.8505, F1: 0.8821
Epoch 19/75, Loss: 0.1327, Acc: 0.9555, Prec: 0.9433, Rec: 0.8287, F1: 0.8823
Epoch 20/75, Loss: 0.1236, Acc: 0.9586, Prec: 0.9322, Rec: 0.8567, F1: 0.8929
Epoch 21/75, Loss: 0.1345, Acc: 0.9580, Prec: 0.9349, Rec: 0.8505, F1: 0.8907
Epoch 22/75, Loss: 0.1257, Acc: 0.9617, Prec: 0.9577, Rec: 0.8474, F1: 0.8992
Epoch 23/75, Loss: 0.1236, Acc: 0.9605, Prec: 0.9300, Rec: 0.8692, F1: 0.8986
Epoch 24/75, Loss: 0.1276, Acc: 0.9592, Prec: 0.9444, Rec: 0.8474, F1: 0.8933
Epoch 25/75, Loss: 0.1291, Acc: 0.9517, Prec: 0.9326, Rec: 0.8193, F1: 0.8723
Epoch 26/75, Loss: 0.1187, Acc: 0.9605, Prec: 0.9300, Rec: 0.8692, F1: 0.8986
Epoch 27/75, Loss: 0.1178, Acc: 0.9611, Prec: 0.9609, Rec: 0.8411, F1: 0.8970
Epoch 28/75, Loss: 0.1148, Acc: 0.9630, Prec: 0.9456, Rec: 0.8660, F1: 0.9041
Epoch 29/75, Loss: 0.1037, Acc: 0.9642, Prec: 0.9521, Rec: 0.8660, F1: 0.9070
Epoch 30/75, Loss: 0.1043, Acc: 0.9686, Prec: 0.9656, Rec: 0.8754, F1: 0.9183
Epoch 31/75, Loss: 0.1091, Acc: 0.9630, Prec: 0.9367, Rec: 0.8754, F1: 0.9050
Epoch 32/75, Loss: 0.1004, Acc: 0.9655, Prec: 0.9650, Rec: 0.8598, F1: 0.9094
Epoch 33/75, Loss: 0.1013, Acc: 0.9636, Prec: 0.9398, Rec: 0.8754, F1: 0.9065
Epoch 34/75, Loss: 0.0972, Acc: 0.9642, Prec: 0.9490, Rec: 0.8692, F1: 0.9073
Epoch 35/75, Loss: 0.1095, Acc: 0.9586, Prec: 0.9474, Rec: 0.8411, F1: 0.8911
Epoch 36/75, Loss: 0.1105, Acc: 0.9624, Prec: 0.9516, Rec: 0.8567, F1: 0.9016
Epoch 37/75, Loss: 0.0990, Acc: 0.9617, Prec: 0.9676, Rec: 0.8380, F1: 0.8982
Epoch 38/75, Loss: 0.0948, Acc: 0.9668, Prec: 0.9527, Rec: 0.8785, F1: 0.9141
Epoch 39/75, Loss: 0.0988, Acc: 0.9592, Prec: 0.9384, Rec: 0.8536, F1: 0.8940
Epoch 40/75, Loss: 0.0962, Acc: 0.9630, Prec: 0.9580, Rec: 0.8536, F1: 0.9028
Epoch 41/75, Loss: 0.0961, Acc: 0.9661, Prec: 0.9588, Rec: 0.8692, F1: 0.9118
Epoch 42/75, Loss: 0.1100, Acc: 0.9598, Prec: 0.9416, Rec: 0.8536, F1: 0.8954
Epoch 43/75, Loss: 0.0974, Acc: 0.9693, Prec: 0.9595, Rec: 0.8847, F1: 0.9206
Epoch 44/75, Loss: 0.0940, Acc: 0.9642, Prec: 0.9521, Rec: 0.8660, F1: 0.9070
Epoch 45/75, Loss: 0.1116, Acc: 0.9580, Prec: 0.9262, Rec: 0.8598, F1: 0.8918
Epoch 46/75, Loss: 0.1077, Acc: 0.9598, Prec: 0.9606, Rec: 0.8349, F1: 0.8933
Epoch 47/75, Loss: 0.1027, Acc: 0.9655, Prec: 0.9618, Rec: 0.8629, F1: 0.9097
Epoch 48/75, Loss: 0.0942, Acc: 0.9655, Prec: 0.9375, Rec: 0.8879, F1: 0.9120
Epoch 49/75, Loss: 0.1029, Acc: 0.9611, Prec: 0.9420, Rec: 0.8598, F1: 0.8990
Epoch 50/75, Loss: 0.1059, Acc: 0.9586, Prec: 0.9322, Rec: 0.8567, F1: 0.8929
Epoch 51/75, Loss: 0.0974, Acc: 0.9624, Prec: 0.9516, Rec: 0.8567, F1: 0.9016
Epoch 52/75, Loss: 0.1039, Acc: 0.9636, Prec: 0.9550, Rec: 0.8598, F1: 0.9049
Epoch 53/75, Loss: 0.0874, Acc: 0.9649, Prec: 0.9344, Rec: 0.8879, F1: 0.9105
Epoch 54/75, Loss: 0.0970, Acc: 0.9636, Prec: 0.9582, Rec: 0.8567, F1: 0.9046
Epoch 55/75, Loss: 0.0909, Acc: 0.9624, Prec: 0.9336, Rec: 0.8754, F1: 0.9035
Epoch 56/75, Loss: 0.0934, Acc: 0.9674, Prec: 0.9686, Rec: 0.8660, F1: 0.9145
Epoch 57/75, Loss: 0.0959, Acc: 0.9674, Prec: 0.9498, Rec: 0.8847, F1: 0.9161
Epoch 58/75, Loss: 0.0950, Acc: 0.9655, Prec: 0.9463, Rec: 0.8785, F1: 0.9111
Epoch 59/75, Loss: 0.0923, Acc: 0.9693, Prec: 0.9658, Rec: 0.8785, F1: 0.9201
Epoch 60/75, Loss: 0.1004, Acc: 0.9642, Prec: 0.9371, Rec: 0.8816, F1: 0.9085
Epoch 61/75, Loss: 0.0933, Acc: 0.9636, Prec: 0.9747, Rec: 0.8411, F1: 0.9030
Epoch 62/75, Loss: 0.0892, Acc: 0.9686, Prec: 0.9593, Rec: 0.8816, F1: 0.9188
Epoch 63/75, Loss: 0.0840, Acc: 0.9699, Prec: 0.9723, Rec: 0.8754, F1: 0.9213
Epoch 64/75, Loss: 0.0811, Acc: 0.9718, Prec: 0.9694, Rec: 0.8879, F1: 0.9268
Epoch 65/75, Loss: 0.0866, Acc: 0.9674, Prec: 0.9498, Rec: 0.8847, F1: 0.9161
Epoch 66/75, Loss: 0.0806, Acc: 0.9674, Prec: 0.9559, Rec: 0.8785, F1: 0.9156
Epoch 67/75, Loss: 0.0821, Acc: 0.9718, Prec: 0.9631, Rec: 0.8941, F1: 0.9273
Epoch 68/75, Loss: 0.0789, Acc: 0.9674, Prec: 0.9410, Rec: 0.8941, F1: 0.9169
Epoch 69/75, Loss: 0.0843, Acc: 0.9693, Prec: 0.9444, Rec: 0.9003, F1: 0.9219
Epoch 70/75, Loss: 0.0877, Acc: 0.9674, Prec: 0.9529, Rec: 0.8816, F1: 0.9159
Epoch 71/75, Loss: 0.0854, Acc: 0.9674, Prec: 0.9753, Rec: 0.8598, F1: 0.9139
Epoch 72/75, Loss: 0.0766, Acc: 0.9693, Prec: 0.9533, Rec: 0.8910, F1: 0.9211
Epoch 73/75, Loss: 0.0833, Acc: 0.9718, Prec: 0.9726, Rec: 0.8847, F1: 0.9266
Epoch 74/75, Loss: 0.0855, Acc: 0.9674, Prec: 0.9410, Rec: 0.8941, F1: 0.9169
Epoch 75/75, Loss: 0.0849, Acc: 0.9668, Prec: 0.9467, Rec: 0.8847, F1: 0.9147

Test Metrics -> Acc: 0.9830, Prec: 0.9727, Rec: 0.9469, F1: 0.9596

batch_size= 64
input_dim = X_train_tensor.shape[1]
hidden_dims = [512,256,128, 64, 32]
dropout = 0.2
lr = 0.001
epochs = 75

13 saniye

"""